## 1. Basic Capacity Expansion Problem

**Objective**: minimizing the cost of fixed and variable costs across a fleet of power generators to meet anticipated future electricity demand.  
**Note**:
Inter-temporal operating constraints (e.g. ramp limits), energy storage, network constraints and geospatial tradeoffs, unit commitment decisions for thermal generators and operating reserves are ignored here

#### Different complexity levels:  
- Optimal thermal generator capacity expansion problem (Greenfield)  
- Co-optimizing thermal generators and variable renewables  
- Brownfield expansion and retirement decisions

$$G = \text{set of generators [geo, coal, CCGT, CT] }$$  
$$H = \text{the set of hours of the year}$$

#### e.g.: Optimal thermal generator capacity expansion problem (Greenfield)

Objective Function and corresponding constraints:  
$$\begin{aligned}
    \min &\sum_{g \in G} \left(FixedCost_g \times CAP_g +  \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)& \\
    &\quad\quad+ \sum_{h \in H} NSECost \times NSE_h & \\
    &\text{s.t.} & \\
    & \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H \\
    & GEN_{g,h} \leq CAP_g & \forall \quad g \in G \text{ \& } h \in H \\
    & CAP_{g} \geq 0 & \forall \quad g \in G \\
    & NSE_{h} \geq 0 & \forall \quad h \in H \\
    & GEN_{g,h} \geq 0 & \forall \quad g \in G \text{ \& } h \in H \\
    \end{aligned}$$
 

Decision Variables:  
$CAP_g$: the capacity (in MW) of each generation type  
$GEN_{g,h}$ : the generation (in MWh) produced by each generator in each hour  
$NSE_h$ :the quantity of involuntarily curtailed demand in each hour (in MWh)

In [1]:
import pandas as pd
df_generators=pd.read_csv('generators.csv', sep=',')
df_generators.fillna(0, inplace=True)

df_variables = df_generators.loc[df_generators.is_variable==1]
df_non_variables = df_generators.loc[df_generators.is_variable!=1]

df_generators

,Resource,Pmax,Pmin,is_variable,CF,VarOM,FuelCost,HeatRate,Capex
0,biomass,20,0.7,0,1.0,5,1,12.76,1_000_000
1,wind_onshore,200,0.0,1,0.3,0,0,0.00,1_000_000
2,wind_offshore,200,0.0,1,0.7,0,0,0.00,1_000_000
3,solar_photovoltaic,100,0.0,1,0.6,0,0,0.00,1_000_000
4,gas_turbine,300,0.7,0,1.0,10,3,9.10,1_000_000
5,pumped_storage,40,0.0,0,1.0,0,1,0.00,1_000_000
6,battery,0,0.0,0,1.0,0,0,0.00,1_000_000


In [2]:
import linopy 

model = linopy.Model()

x = model.add_variables(lower=df_generators.Pmin,upper=df_generators.Pmax, coords=[df_generators.index],name = 'generators')
x


Variable (dim_0: 7)
-------------------
[0]: generators[0] ∈ [0.7, 20]
[1]: generators[1] ∈ [0, 200]
[2]: generators[2] ∈ [0, 200]
[3]: generators[3] ∈ [0, 100]
[4]: generators[4] ∈ [0.7, 300]
[5]: generators[5] ∈ [0, 40]
[6]: generators[6] ∈ [0, 0]

# ... will be completed soon